# SummaryIndex 是否能实现原数据过滤

## 全局设置

In [1]:
%%time

from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding

Settings.llm=OpenAILike(
    # model="qwen2", 
    model="glm4", 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model = OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://ape:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

CPU times: user 3.26 s, sys: 790 ms, total: 4.05 s
Wall time: 5.21 s


## 加载数据

In [4]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

items=[
    "北海公园",
    "故宫",
    "颐和园",
]

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 517 ms, sys: 7.07 ms, total: 524 ms
Wall time: 790 ms


3

In [11]:
documents[0].id_

'https://baike.baidu.com/item/北海公园'

## 索引

In [5]:
%%time

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SummaryIndex

parser = SentenceSplitter(chunk_size=1024,chunk_overlap=20)
nodes = parser.get_nodes_from_documents(documents)

len(nodes)

CPU times: user 39.4 ms, sys: 107 µs, total: 39.5 ms
Wall time: 38.4 ms


59

In [8]:
nodes[2]

TextNode(id_='f11a902a-dfb7-4f1b-afba-c79392041121', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/北海公园', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='438fac785c42c03178c36e55034707ca88d4b78055d6d8f03def3092fff0e665'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='ff7ccd91-5c3b-4907-a8b2-625df201ffeb', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='8ea1a298611e0cf9f2be7a78c781d17c741c891d3e9c6863a227d6a72e367f17'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='23352e71-3e2d-4455-bd40-9cf886078640', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='5454b50965484b92ea5fa47c3159da19a0d6183e771d7f3ff4b4145a808f8dcd')}, text='1949年中华人民共和国成立后，政府拨巨资对北海公园予以修葺，疏浚了湖泊，维修了古建筑，铺设甬道，增设了公共服务设施。 [5]\n1961年北海公园被国务院公布为第一批全国重点文物保护单位。 [4]\n1986年，北海被评为北京十六景之一。 [6]\n2016年，北海公园与仿膳饭庄达成协议，完成漪澜堂古建筑群的腾退工作。为了更好地保护漪澜堂古建筑群的建筑、山

In [9]:
nodes[0]

TextNode(id_='436bbbc3-9ab9-42c2-8df3-f11b2a69dcf9', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='https://baike.baidu.com/item/北海公园', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='438fac785c42c03178c36e55034707ca88d4b78055d6d8f03def3092fff0e665'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='ff7ccd91-5c3b-4907-a8b2-625df201ffeb', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='8ea1a298611e0cf9f2be7a78c781d17c741c891d3e9c6863a227d6a72e367f17')}, text='收藏\n查看我的收藏\n0有用+1\n北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。 [3] [28]\n北海及团城于1961年被中华人民共和国中央人民政府公布为第一批全国重点文物保护单位。 [32]北海公园也是国家AAAA级旅游景区。 [33]\n2023年4月，北海公园入选第二批国家级文明旅游示范单位公示名单。 [37]\n- 中文名\n- 北海公园\n- 外文名\n- Beihai Park\n- 地理位置\n- 北京市西城区文津街1号\n- 气候条件\n- 温带季风气候\n- 开放时间\n- 4月1日-10月31日 06:00-21:00；11月1日-次年3月31日 06:30-20:00 [1]\n- 景点级别\n- 

In [12]:
%%time

summary_index = SummaryIndex(nodes)

CPU times: user 7.24 ms, sys: 0 ns, total: 7.24 ms
Wall time: 6.84 ms


In [13]:
%%time

# 创建索引器

summary_retriever = summary_index.as_retriever()

CPU times: user 38 µs, sys: 7 µs, total: 45 µs
Wall time: 46.3 µs
